<a href="https://colab.research.google.com/github/paslariirina01-lab/git_workhard/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%9F%D0%B0%D1%81%D0%BB%D0%B0%D1%80%D1%8C_%D0%98%D1%80%D0%B8%D0%BD%D0%B0__%22w2v_hw_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [3]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 79.4 MB/s eta 0:00:00


## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [4]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

[==================================================] 100.0% 128.1/128.1MB downloaded
Размер словаря: 400000
Размерность векторов: 100


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

**Базовые операции с векторами**

In [5]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [6]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

Слова, похожие на 'python':
  monty: 0.6886
  php: 0.5865
  perl: 0.5784
  cleese: 0.5447
  flipper: 0.5113


*Ваш ответ здесь*
1. word2vec-google-news-300
Источник данных: новостные статьи Google News.
Объем: примерно 100 млн слов, вектор - 300 измерений.
Зачем может использоваться:классический Word2Vec, подходит для задач семантической близости, поиска аналогий, классификации текста.
2. fasttext-wiki-news-subwords-300
Источник данных: Wikipedia (на английском)
Объем: вектор - 300.
Зачем может использоваться: для работы с редкими словами, морфологией, лемматизацией, классификацией слов.
3. glove-twitter-200
Источник данных: англоязычный твиты из Twitter.
Объем: вектор - 200.
Зачем может использоваться: классификация коротких текстов, работа со сленгом и неформальными выражениями.




**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [7]:
import gensim.downloader as api


model = api.load("word2vec-google-news-300")

print("Размер словаря:", len(model.key_to_index))
print("Размерность векторов:", model.vector_size)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
Размер словаря: 3000000
Размерность векторов: 300


2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [ ]:
def get_similar_words(model, word, topn=10):
    try:
        similar = model.most_similar(word, topn=topn)
        return similar
    except KeyError:
        return f"Слова '{word}' нет в словаре модели."


result = get_similar_words(model, "computer")
print(result)

[('computers', 0.7979379892349243), ('laptop', 0.6640493273735046), ('laptop_computer', 0.6548868417739868), ('Computer', 0.647333562374115), ('com_puter', 0.6082080006599426), ('technician_Leonard_Luchko', 0.5662748217582703), ('mainframes_minicomputers', 0.5617720484733582), ('laptop_computers', 0.5585449934005737), ('PC', 0.5539618730545044), ('maker_Dell_DELL.O', 0.5519254207611084)]


3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [ ]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [ ]:
from gensim.models import Word2Vec


model = Word2Vec(
    sentences=cooking_sentences,
    vector_size=50,     # размер вектора
    window=3,           # размер окна
    min_count=1,        # минимальная частота слова
    workers=2,          # потоков для параллельной обработки
    sg=1                # использовать skip-gram (1 = skip-gram, 0 = CBOW)
)



In [ ]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


4. Проверьте модель

In [ ]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711


In [ ]:
# Найдите слова, похожие на "духовка"

try:
    similar_duhovka = model.wv.most_similar('духовка', topn=5)
    print("\nСлова, похожие на 'духовка':")
    for word, score in similar_duhovka:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

# Найдите слова, похожие на "овощи"

try:
    similar_ovoshchi = model.wv.most_similar('овощи', topn=5)
    print("\nСлова, похожие на 'овощи':")
    for word, score in similar_ovoshchi:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")


Слова, похожие на 'духовка':
  ингредиенты: 0.3199
  десерт: 0.3064
  холодильник: 0.2705
  питание: 0.2243
  пирог: 0.2142

Слова, похожие на 'овощи':
  мариновать: 0.2716
  хлеб: 0.2691
  гриль: 0.2546
  фольга: 0.2409
  сахар: 0.2108


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [ ]:
from gensim.models import FastText


ft_model = FastText(
    sentences=cooking_sentences,  # корпус из задания 3
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

print("Модель FastText обучена!")
print(f"Размер словаря: {len(ft_model.wv.key_to_index)}")

Модель FastText обучена!
Размер словаря: 65


6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [ ]:
try:
    similar_varit = ft_model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar_varit:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

try:
    similar_duhovka = ft_model.wv.most_similar('духовка', topn=5)
    print("\nСлова, похожие на 'духовка':")
    for word, score in similar_duhovka:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

try:
    similar_ovoshi = ft_model.wv.most_similar('овощи', topn=5)
    print("\nСлова, похожие на 'овощи':")
    for word, score in similar_ovoshi:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")

Слова, похожие на 'варить':
  жарить: 0.5353
  парить: 0.4805
  месить: 0.3541
  тушить: 0.3405
  специи: 0.2622

Слова, похожие на 'духовка':
  взбивать: 0.4565
  лимон: 0.3561
  салат: 0.3050
  курица: 0.3041
  тост: 0.2944

Слова, похожие на 'овощи':
  жарить: 0.2960
  фольга: 0.2574
  морковь: 0.2297
  соус: 0.2172
  торт: 0.2094


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [ ]:
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    # Word2Vec
    try:
        w2v_similar = model.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    # FastText
    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

# Сравниваем для разных слов
compare_models('learning')
compare_models('neural')
compare_models('варит')    # опечатка: нет буквы "ь"
compare_models('дуохвка')  # перестановка букв
compare_models('овоши')    # ошибка при написании


Сравнение для слова: 'learning'
  Word2Vec: слово не найдено
  FastText: ['духовка', 'пирог']

Сравнение для слова: 'neural'
  Word2Vec: слово не найдено
  FastText: ['мука', 'травы']

Сравнение для слова: 'варит'
  Word2Vec: слово не найдено
  FastText: ['варить', 'картофель']

Сравнение для слова: 'дуохвка'
  Word2Vec: слово не найдено
  FastText: ['смешивать', 'мука']

Сравнение для слова: 'овоши'
  Word2Vec: слово не найдено
  FastText: ['чай', 'яичница']


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [ ]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']


In [ ]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [ ]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [ ]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


8. Сравните схожесть doc_2 и doc_4

In [ ]:
compare_documents(2, 4)


Схожесть doc_2 и doc_4: -0.0362
  doc_2: python programming for data science
  doc_4: computer vision processes images


9. Найдите самый похожий документ на doc_1

In [ ]:
most_similar_to_doc1 = doc_model.dv.most_similar("doc_1", topn=1)

doc_tag, score = most_similar_to_doc1[0]
doc_id = int(doc_tag.split("_")[1])

print(f"Самый похожий документ на doc_1: {doc_tag} (схожесть {score:.4f})")
print("Текст документа:")
print(documents[doc_id])

Самый похожий документ на doc_1: doc_0 (схожесть 0.2735)
Текст документа:
machine learning is interesting


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [ ]:
from gensim.models import Word2Vec

dimensions = [10, 50, 100]
w2v_models = {}

for dim in dimensions:
    model_temp = Word2Vec(
        sentences=cooking_sentences,
        vector_size=dim,
        window=3,
        min_count=1,
        workers=2,
        sg=1
    )
    w2v_models[dim] = model_temp
    print(f"Обучена модель Word2Vec с размерностью {dim}")

# сравнение похожих слов
def test_similar_words(models, word):
    print(f"\n=== Проверка слова: '{word}' ===")
    for dim, m in models.items():
        try:
            sim = m.wv.most_similar(word, topn=3)
            print(f"Размерность {dim}: {[w for w, _ in sim]}")
        except KeyError:
            print(f"Размерность {dim}: слово '{word}' не найдено")

# пример похожих слов
test_similar_words(w2v_models, "варить")
test_similar_words(w2v_models, "овощи")
test_similar_words(w2v_models, "мясо")

Обучена модель Word2Vec с размерностью 10
Обучена модель Word2Vec с размерностью 50
Обучена модель Word2Vec с размерностью 100

=== Проверка слова: 'варить' ===
Размерность 10: ['рыба', 'сковорода', 'хлеб']
Размерность 50: ['вино', 'ингредиенты', 'хлеб']
Размерность 100: ['чашка', 'вино', 'сковорода']

=== Проверка слова: 'овощи' ===
Размерность 10: ['жарить', 'фольга', 'горшок']
Размерность 50: ['мариновать', 'хлеб', 'гриль']
Размерность 100: ['взбивать', 'питание', 'бекон']

=== Проверка слова: 'мясо' ===
Размерность 10: ['жарить', 'курица', 'мука']
Размерность 50: ['завтрак', 'взбивать', 'жарить']
Размерность 100: ['тост', 'кипятить', 'помидоры']
